Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

1) Download and extract https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

2) python packages needed: compress_pickle[lz4] pandas numpy yahooquery seaborn

2) From the stock dataset, copy "Stock_List.csv" and "Stocks/*" to "data/". You can delete the other files as they are not used (at least yet).

In [4]:
# 

In [5]:
try:
    import pandas as pd
    import numpy as np
    import seaborn
    from glob import glob
    from compress_pickle import dump, load
    import os
    from yahooquery import Ticker
    import timeit
    import time
    import datetime
except ModuleNotFoundError as e:
    print(e)
    print('Please install the missing module(s)')
    print("pip install compress_pickle[lz4] pandas numpy yahooquery seaborn")
    
from get_stock_search_terms import aquire_stock_search_terms as aquire_terms
from get_macroeconomic_data import get_macroeconomic_data as macro_data

In [6]:
# Gather the company info for all the ticker symbols and return a dataframe with relevant search terms for each company.
search_terms = aquire_terms('data/Stocks/')
search_terms.data

,Ticker,Company,Top_Executive,Industry,quoteType,sector,first_traded,year_founded
0,A,"Agilent Technologies, Inc.",Michael McMullen,Diagnostics & Research,EQUITY,Healthcare,1999-11-18 08:30:00,1999
1,AA,Alcoa Corporation,Roy Harvey,Aluminum,EQUITY,Basic Materials,-252322200,1888
2,AAC,Ares Acquisition Corporation,David Kaplan,Shell Companies,EQUITY,Financial Services,2021-03-25 08:30:00,2020
3,AACG,ATA Creativity Global,Xiaofeng Ma,Education & Training Services,EQUITY,Consumer Defensive,2008-01-29 08:30:00,1999
4,AACI,Armada Acquisition Corp. I,Stephen Herbert,Shell Companies,EQUITY,Financial Services,2021-11-10 08:30:00,2020
...,...,...,...,...,...,...,...,...
7677,ZVSAW,None,,,EQUITY,,None,
7678,ZWS,Zurn Elkay Water Solutions Corporation,Todd Adams,Pollution & Treatment Controls,EQUITY,Industrials,2012-03-29 08:30:00,2006
7679,ZYME,Zymeworks Inc.,Ali Tehrani PhD,Biotechnology,EQUITY,Healthcare,2017-04-28 08:30:00,2003
7680,ZYNE,"Zynerba Pharmaceuticals, Inc.",Armando Anido MBA,Drug Manufacturers—Specialty & Generic,EQUITY,Healthcare,2015-08-05 08:30:00,2007


Datasets:
We need a historical dataset of stock prices.
Secondly, we need a dataset of web trends for the same time period. 

On the other hand, once the model is trained, it needs to be able to aquire live data that looks the same as the training data.

Possible sources:
   - twitter
   - google trends

Possible models:
    - Train a model to process text into an encoding
    - Train a model to do stock price prediction
    - Combine the two models to predict stock price/volatility (not sure which is best) from text with greater accuracy